In [1]:
from keras.preprocessing import text
norm_bible="Hello world "
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(norm_bible)

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

vocab_size = len(word2id) + 1 
embed_size = 100

wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in norm_bible]
print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])




Using TensorFlow backend.
Vocabulary Size: 8
Vocabulary Sample: [('l', 1), ('o', 2), ('h', 3), ('e', 4), ('w', 5), ('r', 6), ('d', 7)]


In [2]:


from keras.preprocessing.sequence import skipgrams

# generate skip-grams
skip_grams = [skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in wids]

# view sample skip-grams
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          id2word[pairs[i][0]], pairs[i][0], 
          id2word[pairs[i][1]], pairs[i][1], 
          labels[i]))

IndexError: list index out of range

In [2]:
#https://stackoverflow.com/questions/53503049/measure-similarity-between-two-documents-using-doc2vec
#Model downloaded from:
#https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

from gensim.models import doc2vec
import gensim.downloader as api

from scipy import spatial

model_file= "GoogleNews-vectors-negative300.bin.gz"
d2v_model = doc2vec.Doc2Vec.load(model_file)

fisrt_text = '..'
second_text = '..'

vec1 = d2v_model.infer_vector(fisrt_text.split())
vec2 = d2v_model.infer_vector(second_text.split())

cos_distance = spatial.distance.cosine(vec1, vec2)

UnpicklingError: invalid load key, '3'.

In [ ]:
#TODO: https://towardsdatascience.com/detecting-document-similarity-with-doc2vec-f8289a9a7db7
